# Week 1 — Interactive Quant Risk & Optimization (v4.2 Educational Edition, Live Data)

Welcome! This **hands-on notebook** explains both the *math* and the *intuition* of portfolio risk and optimization.

### What you’ll learn
1. **Measure risk**: volatility, Sharpe, VaR, CVaR, drawdowns
2. **Optimize portfolios**: Equal-Weight vs. Mean–Variance (risk aversion $\gamma$)
3. **Validate & interpret**: cumulative & drawdown curves, rolling Sharpe
4. **Explore interactively**: sliders for VaR confidence (α) and risk aversion (γ)

### Flow of the notebook
**Data → Returns → Risk → Optimization → Backtest → Interpretation**


## 0) Setup & Helpers
This section prepares the plotting style, optional libraries, and utility functions.

**Why these choices?**
- **Matplotlib** is reliable for static figures; **Plotly** adds zoom/hover when available.
- **ipywidgets** enables interactive sliders for teaching intuition.
- We include **quick-export** so you can save figures into `reports/` for slides.


In [ ]:
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (9,4)
plt.rcParams['axes.grid'] = True

# Optional libs (Plotly & ipywidgets): detected at runtime
try:
    import plotly.express as px
    _HAVE_PLOTLY = True
except Exception:
    _HAVE_PLOTLY = False

try:
    import ipywidgets as widgets
    from IPython.display import display
    _HAVE_WIDGETS = True
except Exception:
    _HAVE_WIDGETS = False

# === Figure export helper ===
def export_fig(fig=None, name='chart', fmt='png', out_dir='reports'):
    """Save the current or provided matplotlib figure to reports/ as PNG/PDF."""
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    if fig is None:
        fig = plt.gcf()
    path = Path(out_dir) / f"{name}.{fmt}"
    fig.savefig(path, bbox_inches='tight', dpi=300)
    print(f" Exported chart to {path}")

# === Summary stats under charts ===
def show_summary_stats(returns_series: pd.Series):
    r = returns_series.dropna()
    ann_ret = r.mean()*252
    ann_vol = r.std()*np.sqrt(252)
    sharpe  = ann_ret/(ann_vol+1e-12)
    cum = (1+r).cumprod()
    mdd = (cum/cum.cummax()-1.0).min()
    print(f" Annualized Return: {ann_ret:.2%}")
    print(f" Volatility: {ann_vol:.2%}")
    print(f" Sharpe Ratio: {sharpe:.2f}")
    print(f" Max Drawdown: {mdd:.2%}")

# === Rolling Sharpe ===
def rolling_sharpe(r: pd.Series, window=252):
    mu = r.rolling(window).mean()
    sd = r.rolling(window).std()
    return (np.sqrt(252)*mu)/(sd+1e-12)

# === Simple backtest: monthly rebalancing, transaction costs (bps) ===
def rebalance_backtest(returns: pd.DataFrame, weight_fn, rebalance_freq='M', tx_bps: float = 5.0):
    idx = returns.resample(rebalance_freq).last().index
    weights_rec, port_rets = [], []
    prev_w = None
    for i, end in enumerate(idx):
        window = returns.loc[:end]
        if window.empty: continue
        w = weight_fn(window)
        w = w/(w.sum()+1e-12)
        weights_rec.append((end, w))
        nxt = idx[i+1] if i+1 < len(idx) else returns.index[-1]
        seg = returns.loc[(returns.index> end) & (returns.index<=nxt)]
        if seg.empty: continue
        pr = seg @ w
        if prev_w is not None and tx_bps>0:
            turnover = np.abs(w - prev_w).sum()
            if len(pr)>0:
                pr.iloc[0] -= (tx_bps/10000.0)*turnover
        port_rets.append(pr)
        prev_w = w
    if not port_rets:
        raise ValueError('No backtest segments computed')
    r = pd.concat(port_rets).sort_index()
    curve = (1+r).cumprod()
    return {'returns': r, 'curve': curve, 'weights': weights_rec}

# === Mean–Variance weights via risk aversion γ (SLSQP fallback) ===
from scipy.optimize import minimize
def mv_weights_gamma(mu: np.ndarray, Sigma: np.ndarray, gamma: float = 5.0, long_only=True):
    n = len(mu)
    def obj(w): return -(w@mu - gamma*(w@Sigma@w))
    cons = ({'type':'eq','fun':lambda w: np.sum(w)-1.0},)
    bnds = [(0,1)]*n if long_only else None
    res = minimize(obj, x0=np.ones(n)/n, bounds=bnds, constraints=cons, method='SLSQP')
    return res.x

# === Risk measures (historical VaR / CVaR) ===
def var_historical(series: pd.Series, alpha=0.05):
    return -np.quantile(series.dropna(), alpha)
def cvar_empirical(series: pd.Series, alpha=0.05):
    q = np.quantile(series.dropna(), alpha)
    tail = series[series <= q]
    return -tail.mean()


## 1) Data & Returns — Live Download with Robust Fallback
**We fetch Adjusted Close prices from Yahoo Finance** and compute **log returns**.

### Why log returns?
$r_t = \ln(P_t/P_{t-1})$ are **additive** over time, symmetric for small changes, and often closer to Normal.
Annualization uses **252 trading days** for daily data.


In [ ]:
import pandas as pd
tickers = ['AAPL','MSFT','AMZN','GOOG','META','SPY']
start_date = '2016-01-01'

try:
    import yfinance as yf
    px = yf.download(tickers, start=start_date, progress=False)['Adj Close']
    if isinstance(px.columns, pd.MultiIndex):
        px = px.droplevel(0, axis=1)
    px = px.dropna()
    source = 'Yahoo Finance'
except Exception as e:
    # Synthetic fallback (so the lab is always runnable)
    dates = pd.date_range(start_date, periods=2200, freq='B')
    rng = np.random.default_rng(42)
    rets_syn = pd.DataFrame(rng.normal(0.0004, 0.015, size=(len(dates), len(tickers))), index=dates, columns=tickers)
    px = (1+rets_syn).cumprod()
    source = f'Synthetic (fallback due to: {e})'

def log_returns(prices: pd.DataFrame):
    return np.log(prices/prices.shift(1)).dropna()

R = log_returns(px)
mu_ann = R.mean()*252
cov_ann = R.cov()*252
px.tail(), source

### Interpreting the data
- **Tech-heavy set** (AAPL, AMZN, GOOG, META) will often be **correlated**, so diversification *within* tech is limited.
- **SPY** (broad market ETF) provides an anchor for market risk.


## 2) Baseline Risk Metrics & Tails
**What we measure**
- **Volatility (σ):** dispersion of returns
- **Sharpe Ratio:** reward per unit of risk, $\frac{E[r]}{\sigma}$ (use excess over $r_f$ if available)
- **VaR (α):** loss threshold not exceeded α% of the time
- **CVaR (α):** average loss *beyond* VaR — *severity* of tail events

**Regulatory context:** Basel frameworks often use **99%** VaR/CVaR; discretionary PMs may use **95%**.


In [ ]:
# Equal-Weight baseline (naïve diversification)
w_ew = np.ones(len(mu_ann))/len(mu_ann)
r_ew = R @ w_ew
sharpe_ew = np.sqrt(252)*r_ew.mean()/ (r_ew.std()+1e-12)
vaR5 = var_historical(r_ew, 0.05)
cvaR5 = cvar_empirical(r_ew, 0.05)
pd.Series({'Sharpe_EW':sharpe_ew, 'VaR5%':vaR5, 'CVaR5%':cvaR5})

###  Understanding Return Distributions
- Histogram shows frequency of returns; **fat tails** imply more extreme losses than Normal.
- QQ-plot compares empirical quantiles to Normal — **curved tails** signal non-Normality.


In [ ]:
ax = R['AAPL'].hist(bins=50)
plt.title('AAPL — Daily Returns Histogram')
plt.show()

import statsmodels.api as sm
sm.qqplot(R['AAPL'], line='s')
plt.title('AAPL — QQ Plot (Normal vs Empirical)')
plt.show()

## 3) Portfolio Optimization — Mean–Variance vs Equal-Weight
**Theory recap**
\[ \max_w \; \mu^\top w - \gamma \, w^\top \Sigma w \]
- $\mu$: expected returns; $\Sigma$: covariance matrix
- $\gamma$: **risk aversion** — higher ⇒ more penalty on volatility

**Intuition:** EW assumes all assets are equally attractive; MV uses *information* in $\mu$ and $\Sigma$ to tilt weights.


In [ ]:
assets = mu_ann.index.tolist()
gamma = 5.0  # adjust below with a slider
w_mv = mv_weights_gamma(mu_ann.values, cov_ann.values, gamma=gamma, long_only=True)
pd.Series(w_mv, index=assets, name=f'w_MV_gamma={gamma}').sort_values(ascending=False)

## 4) Backtest — Monthly Rebalance, Transaction Costs
**What is backtesting?** Simulating historical performance with realistic assumptions.

**Why monthly?** Reasonable balance between responsiveness and transaction costs.
**Costs** measured in **bps** (basis points): 5 bps = 0.05% applied at rebalance.


In [ ]:
def wf_ew(window):
    return np.ones(len(assets))/len(assets)

def wf_mv(window):
    r = np.log(window/window.shift(1)).dropna()
    mu_w = r.mean()*252
    cov_w = r.cov()*252
    return mv_weights_gamma(mu_w.values, cov_w.values, gamma=gamma, long_only=True)

bt_ew = rebalance_backtest(R[assets], wf_ew, rebalance_freq='M', tx_bps=5)
bt_mv = rebalance_backtest(R[assets], wf_mv, rebalance_freq='M', tx_bps=5)
bt_ew['curve'].tail(), bt_mv['curve'].tail()

### 4.1 Cumulative & Drawdown Charts — How to Read Them
- **Cumulative growth** shows wealth over time; higher is better, but beware of volatility.
- **Drawdown** is peak-to-trough loss; shallower drawdowns = better downside protection.


In [ ]:
curves = {'EW': bt_ew['curve'], f'MV(γ={gamma})': bt_mv['curve']}
if _HAVE_PLOTLY:
    df = pd.concat(curves, axis=1)
    df.columns.name = 'Strategy'
    melted = df.reset_index().melt(id_vars=['index'], var_name='Strategy', value_name='Growth')
    fig = px.line(melted, x='index', y='Growth', color='Strategy', title='Cumulative Performance — Interactive')
    fig
else:
    for k,v in curves.items(): plt.plot(v, label=k)
    plt.title('Cumulative Performance')
    plt.legend(); plt.show()
    export_fig(name='cumulative_performance', fmt='png', out_dir='reports')

def dd(s): return s/s.cummax()-1.0
if _HAVE_PLOTLY:
    dd_df = pd.concat({k: dd(v) for k,v in curves.items()}, axis=1)
    dd_df.columns.name = 'Strategy'
    ddm = dd_df.reset_index().melt(id_vars=['index'], var_name='Strategy', value_name='Drawdown')
    fig2 = px.line(ddm, x='index', y='Drawdown', color='Strategy', title='Drawdown — Interactive')
    fig2
else:
    for k,v in curves.items(): plt.plot(dd(v), label=f'{k} DD')
    plt.title('Drawdown Comparison')
    plt.legend(); plt.show()


### 4.2 Rolling Sharpe (252-day window)
**Why it matters:** Captures *stability* of performance. Persistent values above 1.0 generally indicate robust alpha.


In [ ]:
rs = rolling_sharpe(bt_mv['returns'], window=252)
plt.plot(rs); plt.title('Rolling Sharpe (MV, 252-day)'); plt.show()
export_fig(name='rolling_sharpe_mv_252', fmt='png', out_dir='reports')
show_summary_stats(bt_mv['returns'])

## 5) Interactive Widgets — Build Intuition Live
These controls let you *see* how risk settings change outcomes, without re-running the entire notebook.


###  VaR Confidence Level (α)
- **What it does:** Changes the probability threshold for extreme losses.
- **How to interpret:** Smaller α (e.g., 1–5%) focuses on *rarer, larger* losses.
- **Regulatory context:** VaR at 99% is common in banking risk.


In [ ]:
if _HAVE_WIDGETS:
    alpha = widgets.FloatSlider(value=0.05, min=0.01, max=0.20, step=0.01,
                                description='VaR α', style={'description_width':'initial'},
                                tooltip='Adjusts VaR confidence; smaller α = rarer, larger tail risk')
    def plot_var(a):
        thr = np.quantile(bt_mv['returns'].dropna(), a)
        plt.figure(figsize=(7,3))
        bt_mv['returns'].hist(bins=50)
        plt.axvline(thr, linestyle='--', label=f'VaR {a*100:.1f}%')
        plt.legend(); plt.title('Interactive Historical VaR (MV)'); plt.show()
    out = widgets.interactive_output(plot_var, {'a': alpha})
    display(alpha); display(out)
else:
    print('ipywidgets not available — install and reload the notebook to enable sliders.')

###  Risk Aversion (γ) — Impact on Weights
- **Low γ (≈1–3):** risk-tolerant → more weight to higher-return, higher-volatility assets
- **High γ (≥10):** risk-averse → more diversified, lower-volatility allocations
**Tip:** Watch how weights rotate from growth names to broader market exposure as γ increases.


In [ ]:
if _HAVE_WIDGETS:
    gamma_slider = widgets.FloatSlider(value=5.0, min=0.1, max=25.0, step=0.1,
                                       description='γ (risk aversion)', style={'description_width':'initial'},
                                       tooltip='Higher γ penalizes variance more strongly')
    def solve_and_plot(g):
        w = mv_weights_gamma(mu_ann.values, cov_ann.values, gamma=g, long_only=True)
        plt.figure(figsize=(9,3))
        plt.bar(assets, w)
        plt.xticks(rotation=45, ha='right'); plt.title(f'MV Weights (γ={g:.1f})')
        plt.tight_layout(); plt.show()
    out2 = widgets.interactive_output(solve_and_plot, {'g': gamma_slider})
    display(gamma_slider); display(out2)
else:
    print('ipywidgets not available — install and reload the notebook to enable sliders.')

## 6) Interpreting Results & What to Try Next
**What you learned**
- Quantified portfolio risk (σ, Sharpe, VaR, CVaR) and *why tails matter*
- How risk aversion (γ) translates into portfolio tilts
- How to read cumulative vs drawdown plots and rolling Sharpe

**Next steps**
- Try **CVaR optimization** (tail-risk minimization)
- Add **Black–Litterman** views to bake in your beliefs about relative performance
- Run **factor regressions** (CAPM/FF3/FF5) to separate α (skill) from β (systematic risk)


###  Challenges (optional)
1. Change `rebalance_freq` from `'M'` to `'W'` — does performance improve after costs?
2. Increase transaction costs from 5 to 25 bps — which strategy is more resilient to friction?
3. Replace Mean–Variance with **Equal Risk Contribution** or **CVaR** and compare.
